## 函数装饰器

- 一种修改或增强函数功能的方式，而不改变函数本身代码

In [1]:
# 装饰器本质上是一个函数
# 它接收一个函数作为参数，返回一个新函数

def my_decorator(func):
    def wrapper():
        print("函数执行前")
        func()
        print("函数执行后")
    return wrapper

@my_decorator
def say_hello():
    print("Hello!")

say_hello()

函数执行前
Hello!
函数执行后


In [ ]:
def log_decorator(func):
    def wrapper(*args, **kwargs):
        print(f"调用函数: {func.__name__}")
        print(f"参数: args={args}, kwargs={kwargs}")
        result = func(*args, **kwargs)
        print(f"返回: {result}")
        return result
    return wrapper

@log_decorator
def add(a, b):
    return a + b

print(add(3, 5))

调用函数: add
参数: args=(3, 5), kwargs={}
返回: 8
8


### 保留元信息

In [3]:
from functools import wraps

def preserve_info_decorator(func):
    @wraps(func)  # 保留原函数的元信息
    def wrapper(*args, **kwargs):
        print(f"执行 {func.__name__}")
        return func(*args, **kwargs)
    return wrapper

@preserve_info_decorator
def calculate(x, y):
    """计算两个数的和"""
    return x + y

print(calculate.__name__)  # calculate（不是wrapper）
print(calculate.__doc__)   # 计算两个数的和

calculate
计算两个数的和


### 装饰器堆叠

In [6]:
def decorator1(func):
    def wrapper():
        print("装饰器1开始")
        func()
        print("装饰器1结束")
    return wrapper

def decorator2(func):
    def wrapper():
        print("装饰器2开始")
        func()
        print("装饰器2结束")
    return wrapper

# 从下往上执行
@decorator1
@decorator2
def my_function():
    print("原函数")

my_function()
# 输出:
# 装饰器1开始
# 装饰器2开始
# 原函数
# 装饰器2结束
# 装饰器1结束

# 等价于:
# my_function = decorator1(decorator2(my_function))

装饰器1开始
装饰器2开始
原函数
装饰器2结束
装饰器1结束


### 装饰类

In [ ]:
def log_class(cls):
    """类装饰器，在调用方法前后打印日志"""
    class Wrapper:
        def __init__(self, *args, **kwargs):
            self.wrapped = cls(*args, **kwargs)  # 实例化原始类
        
        def __getattr__(self, name):
            """拦截未定义的属性访问，转发给原始类"""
            return getattr(self.wrapped, name)
        
        def display(self):
            print(f"调用 {cls.__name__}.display() 前")
            self.wrapped.display()
            print(f"调用 {cls.__name__}.display() 后")
    
    return Wrapper  # 返回包装后的类

@log_class
class MyClass:
    def display(self):
        print("这是 MyClass 的 display 方法")

obj = MyClass()
obj.display()

## 类装饰器

### 类装饰函数

In [4]:
class CountCalls:
    """统计函数调用次数"""
    def __init__(self, func):
        self.func = func
        self.count = 0
    
    def __call__(self, *args, **kwargs):
        self.count += 1
        print(f"{self.func.__name__} 被调用第 {self.count} 次")
        return self.func(*args, **kwargs)

@CountCalls
def greet(name):
    print(f"Hello, {name}")

greet("Alice")
greet("Bob")

greet 被调用第 1 次
Hello, Alice
greet 被调用第 2 次
Hello, Bob


### 类装饰类

In [ ]:
class SingletonDecorator:
    """类装饰器，使目标类变成单例模式"""
    def __init__(self, cls):
        self.cls = cls
        self.instance = None
    
    def __call__(self, *args, **kwargs):
        """拦截实例化过程，确保只创建一个实例"""
        if self.instance is None:
            self.instance = self.cls(*args, **kwargs)
        return self.instance

@SingletonDecorator
class Database:
    def __init__(self):
        print("Database 初始化")

db1 = Database()
db2 = Database()
print(db1 is db2)  # True，说明是同一个实例

### 带参数的类装饰器

In [ ]:
class Retry:
    """带参数的重试装饰器类"""
    def __init__(self, max_attempts=3):
        self.max_attempts = max_attempts
    
    def __call__(self, func):
        def wrapper(*args, **kwargs):
            for attempt in range(self.max_attempts):
                try:
                    return func(*args, **kwargs)
                except Exception as e:
                    if attempt == self.max_attempts - 1:
                        raise e
                    print(f"重试 {attempt + 1}/{self.max_attempts}")
            return None
        return wrapper

@Retry(max_attempts=3)
def unstable_api():
    import random
    if random.random() < 0.8:
        raise ConnectionError("API调用失败")
    return "成功"

print(unstable_api())

## 内置装饰器

### @staticmethod - 静态方法

In [8]:
class MathUtils:
    @staticmethod
    def add(a, b):
        """静态方法，不访问实例属性"""
        return a + b
    
    @staticmethod
    def multiply(a, b):
        return a * b

print(MathUtils.add(3, 4))  # 7
print(MathUtils.multiply(3, 4))  # 12

7
12


### @classmethod - 类方法

In [7]:
class Person:
    count = 0  # 类变量
    
    def __init__(self, name):
        self.name = name
        Person.count += 1
    
    @classmethod
    def get_count(cls):
        """类方法，可以访问类变量"""
        return cls.count
    
    @classmethod
    def create_anonymous(cls):
        """类方法作为工厂方法"""
        return cls("Anonymous")

p1 = Person("Alice")
p2 = Person("Bob")
p3 = Person.create_anonymous()

print(Person.get_count())  # 3

3


### @property - 属性装饰器

In [ ]:
class Circle:
    def __init__(self, radius):
        self._radius = radius
    
    @property
    def radius(self):
        """获取半径"""
        print("获取半径")
        return self._radius
    
    @radius.setter
    def radius(self, value):
        """设置半径（带验证）"""
        if value < 0:
            raise ValueError("半径不能为负")
        print(f"设置半径为 {value}")
        self._radius = value
    
    @property
    def area(self):
        """计算面积（只读属性）"""
        return 3.14159 * self._radius ** 2

circle = Circle(5)
print(circle.radius)  # 获取半径，5
circle.radius = 10     # 设置半径为 10
print(circle.area)     # 314.159
# circle.area = 100    # ❌ AttributeError: can't set attribute

### @dataclass (Python 3.7+)

In [ ]:
from dataclasses import dataclass, field
from typing import List

@dataclass
class Person:
    """数据类，自动生成 __init__, __repr__ 等方法"""
    name: str
    age: int
    hobbies: List[str] = field(default_factory=list)
    
    def is_adult(self):
        return self.age >= 18

p1 = Person("Alice", 25, ["reading", "swimming"])
p2 = Person("Bob", 30)

print(p1)  # Person(name='Alice', age=25, hobbies=['reading', 'swimming'])
print(p1.is_adult())  # True